In [1]:
import warnings
warnings.filterwarnings("ignore")

# Training Deep Netural Nets

#### 面对复杂任务可以增加网络深度，网络深度加大会面临:
#### 1.梯度弥散或是梯度爆炸，会影响深度网络的较低层很难训练
#### 2.训练慢
#### 3.过多的参数会面临过拟合问题

### Vanishing/Exploding Gradients Problem
意义：在训练过程中，反向传播往往会使得梯度随着深度的增加会越来越小，导致较低层的网络单元无法很好的训练，并且无法收敛，叫做梯度弥散；同理相反，梯度越来越大，导致很多单元权重会更新很大的，算法也会发散。（通常，深度网络面临不稳定的梯度，不同层以不同的速度学习）

注：对于sigmoid的函数而言，它存在的问题是当函数值在2端时会处于饱和状态，导致梯度为0，从而传递给下层的梯度也随之消失。

###### Xavier and He Initialization
<img src="./pic/Initalization-parameters-for-each-type-of-activation-function.png"/>

In [2]:
import tensorflow as tf

In [3]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [4]:
reset_graph()

In [5]:
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None,n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None,10), name="y")

In [6]:
# tf.variance_scaling_initializer()是He初始化只会考虑fan_in而不是平均，类似于Xavier初始化，
#可以设置为mode="FAN_AVG"
he_init = tf.variance_scaling_initializer()
hidden1 = tf.contrib.layers.fully_connected(X,n_hidden1,weights_initializer=he_init,scope="h1")

### Nonsaturating Activation Functions
1.ReLU替代sigmoid或是双曲线；
公式:$ReLU(z)=max(0,z)$
优点:在为正数的时候不会饱和,且可以加快计算
缺点:会有死亡的ReLUs（在训练过程中，一些神经元死亡了，对外都输出0），如果使用了较大的学习率可能发生半数网络的神经元是死亡的，且死亡了由于梯度为负数导致输出始终是0无法回生。
2.ReLU变体一leaky ReLU：
公式：$LeakyReLU_α(z)=max(αz,z)$
<img src="./pic/Leaky-ReLU.png"/>
3.ReLU变体二exponential linear unit(ELU)
公式:$$ ELU_α(z) =\left\{
\begin{array}{rcl}
α(exp(z) - 1)     &      z < 0 \\
z & z >=0
\end{array} \right. $$
<img src="./pic/ELU.png"/>
不同点:当z<0时，允许单元还有平均输出接近于0，可以缓解梯度弥散问题。超参数α可以调整为任何值；对于z<0的梯度不为0，避开了死亡神经元；处处可导，在z=0时还可以加速梯度下降。
缺点:比ReLU和它的变体计算慢

In [7]:
#tf.nn.leaky_relu
hidden11 = tf.contrib.layers.fully_connected(X,n_hidden1,weights_initializer=he_init,scope="h11",activation_fn=tf.nn.elu)

### Batch Normalization
1.机器学习领域有个很重要的假设：IID独立同分布假设，就是假设训练数据和测试数据是满足相同分布的，这是通过训练数据获得的模型能够在测试集获得好的效果的一个基本保障。

2.BN就是在深度神经网络训练过程中使得每一层神经网络的输入保持相同分布的。
解决问题:内协变量移位(internal covariate shift)

3.BN的基本思想其实相当直观：<font color=red>因为深层神经网络在做非线性变换前的激活输入值（就是那个x=WU+B，U是输入）随着网络深度加深或者在训练过程中，其分布逐渐发生偏移或者变动，之所以训练收敛慢，一般是整体分布逐渐往非线性函数的取值区间的上下限两端靠近</font>（对于Sigmoid函数来说，意味着激活输入值WU+B是大的负值或正值），所以这导致反向传播时低层神经网络的梯度消失，这是训练深层神经网络收敛越来越慢的本质原因，<font color=red>而BN就是通过一定的规范化手段，把每层神经网络任意神经元这个输入值的分布强行拉回到均值为0方差为1的标准正态分布</font>，其实就是把越来越偏的分布强制拉回比较标准的分布，这样使得激活输入值落在非线性函数对输入比较敏感的区域，这样输入的小变化就会导致损失函数较大的变化，意思是这样让梯度变大，避免梯度消失问题产生，而且梯度变大意味着学习收敛速度快，能大大加快训练速度。一句话:<font color=red>对于每个隐层神经元，把逐渐向非线性函数映射后向取值区间极限饱和区靠拢的输入分布强制拉回到均值为0方差为1的比较标准的正态分布，使得非线性变换函数的输入值落入对输入比较敏感的区域，以此避免梯度消失问题。</font>因为梯度一直都能保持比较大的状态，所以很明显对神经网络的参数调整效率比较高，就是变动大，就是说向损失函数最优值迈动的步子大，也就是说收敛地快。BN说到底就是这么个机制，方法很简单，道理很深刻。

4.Batch Normalization algotithm
###### 4.1.$μ_B = \frac{1}{m_B}\sum^{m_B}_{i=1}x^i$ ----期望，m为本次mini-batch的样例数量
###### 4.2.$σ^2_B = \frac{1}{m_B}\sum^{m_B}_{i=1}(x^{(i)} - μ_B)^2$ ----方差
###### 4.3.$X^{(i)} = \frac{x^{(i)} - μ_B}{\sqrt{σ^2_B + ε}}$ ----0中心标准化值，ε用来避开除0的一个小数（$10^{-3}$）
###### 4.4.$z^{(i)} = γX^{(i)}+β$ ----γ是本层的缩放参数，β本层的位移参数,$z^{(i)}$是BN的输出

5.可以一定程度减少梯度弥散问题，减少对于网络初始化的敏感程度，可以使用更大的学习率加快学习过程；它会增加一定的复杂性，使得每层网络的学习变慢(对于一个需要快速预测的网络，优先考虑ELU和He初始化)

<img src='./pic/BN.png'/>
这里的标准化可以放在激活函数前也可以放在激活函数后

In [8]:
reset_graph()

In [9]:
is_training = tf.placeholder(tf.bool,shape=(),name="is_training")
bn_params = {
    'is_training' : is_training,
    'decay' : 0.99,
    'updates_collections' : None
}

In [11]:
reset_graph()

import tensorflow as tf

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

training = tf.placeholder_with_default(False, shape=(), name='training')

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
bn1_act = tf.nn.elu(bn1)

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = tf.layers.batch_normalization(logits_before_bn, training=training,
                                       momentum=0.9)